## MODELOS

#### PlanTL-GOB-ES/roberta-base-bne-sqac

In [35]:
%pip install transformers torch tqdm

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


CONFIG

In [36]:
from pathlib import Path
from collections import deque
from datetime import datetime as dt
import json
import re
import pprint

# Parámetros con preguntas
QUESTIONS = {
    "precio": "¿Cuál es el precio del producto?",
    "nombre": "¿Cuál es el nombre del producto?",
    "marca": "¿Cuál es la marca del producto?",
    "unidad": "¿En qué unidad se vende el producto?",
    "precio_unidad_basica": "¿Cuál es el precio por unidad básica?",
}

MAX_ATTEMPTS = 5
NEED_MATCHES = 2

CARGA DEL MODELO

In [37]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering

MODEL_ID = "PlanTL-GOB-ES/roberta-base-bne-sqac"

def load_qa_model(model_id: str = MODEL_ID):
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    model = AutoModelForQuestionAnswering.from_pretrained(model_id, device_map="auto")
    return pipeline("question-answering", model=model, tokenizer=tokenizer)

qa_pipe = load_qa_model()

Device set to use cpu


LIMPIEZA

In [38]:
import html as html_lib

TAG_RE = re.compile(r"<[^>]+>")
SCRIPT_RE = re.compile(r"<(script|style).*?>.*?</\1>", re.I | re.S)
WS_RE = re.compile(r"\s+")
BODY_RE = re.compile(r"<body[^>]*>(.*?)</body>", re.I | re.S)

def extract_body_content(raw_html: str):
    match = BODY_RE.search(raw_html)
    return match.group(1).strip() if match else raw_html

def strip_html(raw: str) -> str:
    raw = SCRIPT_RE.sub(" ", raw)
    raw = TAG_RE.sub(" ", raw)
    return WS_RE.sub(" ", html_lib.unescape(raw)).strip()

PREGUNTA

In [39]:
def ask_model(context: str):
    """Devuelve un dict con todas las respuestas de QA"""
    respuestas = {}
    for key, pregunta in QUESTIONS.items():
        result = qa_pipe(question=pregunta, context=context)
        answer = result.get("answer", "").strip()
        if not answer or answer.lower() in {"no", "ninguno"}:
            answer = "NA"
        respuestas[key] = answer
        
    if any(value == "NA" for value in respuestas.values()):
        return None
    return respuestas

FORMATEAR (HTML - TXT)

In [40]:
text_path = Path("data/processed/Brazil/1_extracted.txt")
html_path = Path("data/raw/Mexico/Alsuper/Arroz/Verde Valle/1.html")

def procesar_archivos_reales(url:str, retail: str, country: str):
    resultado_final = {
        "HTML": {},
        "TXT": {}
    }

    # 📥 Procesar HTML
    if html_path.exists():
        raw_html = html_path.read_text(errors="ignore")
        html_content = extract_body_content(raw_html)
        context = strip_html(html_content)

        respuestas = deque(maxlen=NEED_MATCHES)
        for intento in range(1, MAX_ATTEMPTS + 1):
            result = ask_model(context)
            if not result:
                print(f"❌ HTML intento {intento} fallido.")
                continue

            result["url"] = url
            result["retail"] = retail
            result["pais"] = country
            respuestas.append(json.dumps(result, sort_keys=True))

            if len(respuestas) == NEED_MATCHES and len(set(respuestas)) == 1:
                print(f"✅ HTML: respuesta estable (intento {intento})")
                key = result["nombre"].lower().replace(" ", "_")
                resultado_final["HTML"][key] = result
                break
    else:
        print("❌ HTML no encontrado:", html_path)

    # 📝 Procesar TXT
    if text_path.exists():
        context = text_path.read_text(errors="ignore").strip()

        respuestas = deque(maxlen=NEED_MATCHES)
        for intento in range(1, MAX_ATTEMPTS + 1):
            result = ask_model(context)
            if not result:
                print(f"❌ TXT intento {intento} fallido.")
                continue

            result["url"] = url
            result["retail"] = retail
            result["pais"] = country
            respuestas.append(json.dumps(result, sort_keys=True))

            if len(respuestas) == NEED_MATCHES and len(set(respuestas)) == 1:
                print(f"✅ TXT: respuesta estable (intento {intento})")
                key = result["nombre"].lower().replace(" ", "_")
                resultado_final["TXT"][key] = result
                break
    else:
        print("❌ TXT no encontrado:", text_path)

    # Mostrar resultados
    print("\n🔍 Resultado final:\n")
    pprint.pprint(resultado_final)

    return resultado_final

HTML (o texto)

In [41]:
productos = procesar_archivos_reales(url= "Esto es una url", retail="Alsuper", country="Mexico")

✅ HTML: respuesta estable (intento 2)
✅ TXT: respuesta estable (intento 2)

🔍 Resultado final:

{'HTML': {'arroz_salvaje': {'marca': 'Arroz Marca Verde Valle',
                            'nombre': 'Arroz Salvaje',
                            'pais': 'Mexico',
                            'precio': '252246 $38.90',
                            'precio_unidad_basica': '252246 $38.90',
                            'retail': 'Alsuper',
                            'unidad': '252246 $38.90',
                            'url': 'Esto es una url'}},
 'TXT': {'arroz_súper_extra_verde_valle': {'marca': 'Verde Valle',
                                           'nombre': 'Arroz súper extra Verde '
                                                     'Valle',
                                           'pais': 'Mexico',
                                           'precio': '$31.90',
                                           'precio_unidad_basica': '252246\n'
                                            

#### llama\phi-2.Q4_K_M

In [1]:
%pip install transformers torch --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\santi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Config

In [ ]:
import subprocess
from pathlib import Path

LLAMA_CLI_PATH = Path(r"C:\Users\santi\AppData\Local\Microsoft\WinGet\Packages\ggml.llamacpp_Microsoft.Winget.Source_8wekyb3d8bbwe\llama-cli.exe")
MODEL_PATH = Path(r"models/phi-2.Q5_K_S.gguf")

try:
    with open(r'data/raw-cleaned/Mexico/HEB/Arroz/Verde Valle/texto_producto.txt', 'r', encoding='utf-8') as file:
        texto_producto_actual = file.read() # Lee el contenido del archivo texto_producto.txt
except FileNotFoundError:
    print("Error: El archivo 'texto_producto.txt' no se encontró. Asegúrate de que esté en la misma carpeta.")
    texto_producto_actual = ""

try:
    with open('prompt.txt', 'r', encoding='utf-8') as file:
        prompt_template = file.read() # Lee el contenido del archivo prompt.txt
except FileNotFoundError:
    print("Error: El archivo 'prompt.txt' no se encontró. Asegúrate de que esté en la misma carpeta.")
    prompt_template = ""

# Reemplazamos el marcador de posición en el prompt leído con el contenido del producto.
if prompt_template and texto_producto_actual:
    prompt_final = prompt_template.format(PRODUCT_TEXT_PLACEHOLDER=texto_producto_actual)
else:
    prompt_final = "Error al construir el prompt. Revisa los archivos."

# Imprimir el prompt para depuración
#print("📨 Prompt generado (con few-shot):")
#print(prompt_final)

# 🔧 Argumentos de generación (corregidos, sin espacios extras)
args = [
    str(LLAMA_CLI_PATH),
    "-m", str(MODEL_PATH),
    "-p", prompt_final,
    "-n", "256",
    "--temp", "0.5",
    "--top-p", "0.9",
    "--repeat-penalty", "1.2"
]

# ▶️ Ejecutar el comando
try:
    result = subprocess.run(
        args,
        capture_output=True,
        text=True,
        check=True,
        encoding="utf-8"  # Para manejar acentos y caracteres especiales
    )
    output = result.stdout.strip()

    print("📤 RESPUESTA DEL MODELO:")
    print(output)

except subprocess.CalledProcessError as e:
    print("❌ ERROR al ejecutar llama-cli:")
    print(e.stderr if e.stderr else str(e))
except FileNotFoundError as e:
    print("❌ ERROR: No se encontró llama-cli.exe o el modelo.")
    print("Verifica la ruta a 'llama-cli.exe' y 'phi-2.Q4_K_M.gguf'.")

📤 RESPUESTA DEL MODELO:
Instrucción: Extrae la siguiente información del Texto del producto: Tipo de alimento, Precio, Moneda, Marca, Cantidad, Unidad. Responde únicamente con el formato: "Tipo de alimento, Precio, Moneda, Marca, Cantidad, Unidad".

Ejemplo 1:
Texto: Marca: Bimbo COP $7.850 (COP $15.70 / gramo)El Pan Tajado Bimbo Artesano es la elección perfecta para quienes buscan un sabor auténtico y una textura inigualable. Ideal para toda la familia.Presentación en bolsa de 500 gramos.Unidad de Venta: Bolsa 500g
Respuesta: Pan, 7850, COP, Bimbo, 500, gramos
###

Ejemplo 2:
Texto: Consume leche Alpura COP $1800.550 La Leche Entera Alpura es fresca y nutritiva, ideal para toda la familia. Presentación en botella de 1 litro. Unidad de Venta: Botella 1L
Respuesta: Leche, 1800.550, COP, Alpura, 1, litro
###

Texto: S O S Arroz Precocido 900 g
S O S
$31.90
Referencia: 252246
$38.90
.
Obtén un 5% de cashback en todas tus compras en heb.com.mx y nuestra app con la tarjeta de crédito HEB Af

Limpieza

In [ ]:
from pathlib import Path
import re
import html as html_lib


# Extracción de HTML
TAG_RE = re.compile(r"<[^>]+>")
SCRIPT_RE = re.compile(r"<(script|style).*?>.*?</\1>", re.I | re.S)
WS_RE = re.compile(r"\s+")
BODY_RE = re.compile(r"<body[^>]*>(.*?)</body>", re.I | re.S)

def extract_body_content(raw_html: str):
    match = BODY_RE.search(raw_html)
    return match.group(1).strip() if match else raw_html

def strip_html(raw: str) -> str:
    raw = SCRIPT_RE.sub(" ", raw)
    raw = TAG_RE.sub(" ", raw)
    return WS_RE.sub(" ", html_lib.unescape(raw)).strip()

PROMPT

In [ ]:
from transformers import TextStreamer

FIELDS = ["nombre", "precio", "marca", "unidad", "precio_unidad_basica", "url"]

def construir_prompt(texto_producto: str):
    return (
        "<|system|>\n"
        "Eres un sistema extractor de información.\n"
        "A partir del texto del producto, responde los siguientes campos en este orden separados por comas:\n"
        f"{', '.join(FIELDS)}.\n"
        "Si falta un valor, escribe NA.\n"
        "<|user|>\n"
        f"{texto_producto}\n"
        "<|assistant|>\n"
    )



MODELO

In [ ]:
def responder_zephyr(prompt: str, max_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            streamer=TextStreamer(tokenizer),
            do_sample=False,
            temperature=0.0
        )

    respuesta = tokenizer.decode(output[0], skip_special_tokens=True)
    return respuesta[len(prompt):].strip()

PARSEAR 

In [ ]:
def extraer_json(respuesta: str):
    partes = [p.strip() for p in respuesta.split(",")]
    if len(partes) != len(FIELDS):
        print("❌ Respuesta incompleta:", respuesta)
        return None
    return dict(zip(FIELDS, partes))

HTML o TXT

In [ ]:
from pathlib import Path
import html as html_lib
import re

# Helpers HTML
def strip_html(raw: str) -> str:
    TAG_RE = re.compile(r"<[^>]+>")
    SCRIPT_RE = re.compile(r"<(script|style).*?>.*?</\1>", re.I | re.S)
    WS_RE = re.compile(r"\s+")
    raw = SCRIPT_RE.sub(" ", raw)
    raw = TAG_RE.sub(" ", raw)
    return WS_RE.sub(" ", html_lib.unescape(raw)).strip()

def test_archivo(path: Path, tipo: str, retail: str, pais: str):
    if not path.exists():
        print(f"❌ Archivo no encontrado: {path}")
        return None

    texto = path.read_text(errors="ignore")
    if path.suffix == ".html":
        texto = strip_html(texto)

    prompt = construir_prompt(texto)
    respuesta = responder_zephyr(prompt)

    print(f"\n📥 {tipo.upper()} → Prompt output:\n", respuesta)

    parsed = extraer_json(respuesta)
    if parsed:
        parsed["retail"] = retail
        parsed["pais"] = pais
        return parsed
    return None

PROCESAMIENTO

In [ ]:
from pprint import pprint

# Archivos de ejemplo
html_path = Path("data/raw/Mexico/Alsuper/Arroz/Verde Valle/1.html")
text_path = Path("data/processed/Brazil/1_extracted.txt")

retail = "Alsuper"
pais = "Mexico"

resultado = {
    "HTML": {},
    "TXT": {}
}

html_res = test_archivo(html_path, "html", retail, pais)
if html_res:
    key = html_res["nombre"].lower().replace(" ", "_")
    resultado["HTML"][key] = html_res

txt_res = test_archivo(text_path, "txt", retail, pais)
if txt_res:
    key = txt_res["nombre"].lower().replace(" ", "_")
    resultado["TXT"][key] = txt_res

print("\n✅ Resultado final:")
pprint(resultado)

output_path = Path("data/processed/arroz_mistral.json")
output_path.write_text(json.dumps(resultado, ensure_ascii=False, indent=2))
print(f"📦 Guardado en: {output_path}")

#### LIQUID - Text Generation

In [ ]:
%pip install transformers accelerate sentencepiece

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\santi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\santi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
%pip install --upgrade transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\santi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Carga del modelo

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float32,  # O usa bfloat16 en GPU
    device_map="auto"
)

C:\Users\santi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: The checkpoint you are trying to load has model type `lfm2` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

Limpieza Simple

In [ ]:
import re
import html as html_lib

def strip_html(raw: str) -> str:
    TAG_RE = re.compile(r"<[^>]+>")
    SCRIPT_RE = re.compile(r"<(script|style).*?>.*?</\1>", re.I | re.S)
    WS_RE = re.compile(r"\s+")

    raw = SCRIPT_RE.sub(" ", raw)
    raw = TAG_RE.sub(" ", raw)
    return WS_RE.sub(" ", html_lib.unescape(raw)).strip()

Prompt

In [ ]:
CAMPOS = ["nombre", "precio", "marca", "unidad", "precio_unidad_basica"]

def construir_prompt(texto: str) -> str:
    
    return (
        f"A partir del siguiente texto de producto:\n\n{texto}\n\n"
        f"Extrae la siguiente información en este orden, separados por comas:\n"
        f"{', '.join(CAMPOS)}.\n"
        "Si falta un dato, escribe 'NA'. No des explicaciones, solo responde los datos en un renglón."
    )

Llamado del Modelo

In [ ]:
from transformers import TextStreamer
streamer = TextStreamer(tokenizer)

prompt = construir_prompt(texto)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=False,
    temperature=0.0,
    streamer=streamer
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
respuesta = generated_text[len(prompt):].strip()
print("\n🔍 Respuesta del modelo:\n", respuesta)

Format

In [ ]:
def respuesta_a_dict(respuesta: str) -> dict | None:
    partes = [p.strip() for p in respuesta.split(",")]
    if len(partes) != len(CAMPOS):
        print("❌ Respuesta inválida o incompleta:", respuesta)
        return None

    return dict(zip(CAMPOS, partes))
    

HTML / TXT

In [ ]:
from pathlib import Path

def procesar_archivo(path: Path, tipo: str, retail="Jumbo", pais="Colombia") -> dict | None:
    if not path.exists():
        print(f"❌ Archivo no encontrado: {path}")
        return None

    texto = path.read_text(errors="ignore")

    if path.suffix == ".html":
        texto = strip_html(texto)

    prompt = construir_prompt(texto)
    print(f"\n📨 Prompt generado ({tipo}):\n", prompt)

    respuesta = generar_respuesta(prompt)
    print(f"\n🤖 Respuesta del modelo:\n", respuesta)

    info = respuesta_a_dict(respuesta)
    if info:
        info["retail"] = retail
        info["pais"] = pais
    return info

EXECUTE

In [ ]:
from pprint import pprint
import json

# 🚀 Rutas de tus archivos
html_path = Path("data/raw/Mexico/Alsuper/Arroz/Verde Valle/1.html")
txt_path = Path("data/processed/Brazil/1_extracted.txt")

# 🛍️ Metadatos
retail = "Alsuper"
pais = "Mexico"

# 📦 Resultados
resultado_final = {"HTML": {}, "TXT": {}}

# 🔍 HTML
res_html = procesar_archivo(html_path, tipo="HTML", retail=retail, pais=pais)
if res_html:
    key = res_html["nombre"].lower().replace(" ", "_")
    resultado_final["HTML"][key] = res_html

# ✏️ TXT
res_txt = procesar_archivo(txt_path, tipo="TXT", retail=retail, pais=pais)
if res_txt:
    key = res_txt["nombre"].lower().replace(" ", "_")
    resultado_final["TXT"][key] = res_txt

# 👀 Mostrar en pantalla
print("\n✅ Resultado final:")
pprint(resultado_final)

In [ ]:
from datetime import datetime

out_path = Path("data/processed/productos_extraidos.json")
out_path.parent.mkdir(parents=True, exist_ok=True)

with out_path.open("w", encoding="utf-8") as f:
    json.dump(resultado_final, f, ensure_ascii=False, indent=2)

print(f"\n💾 Resultado guardado en: {out_path}")